In [1]:
import os
import random
import json
import string

In [2]:
# 获取谣言原文和非谣言原文和谣言评论和非谣言评论文件的全部路径
def get_data_dir(target_path):
    rumor_class_dirs = os.listdir(target_path)

    rumor_file_dir_list = [] #谣言数据根目录
    non_rumor_file_dir_list = [] #非谣言数据根目录

    #解析谣言和非谣言的数据目录
    for filename in rumor_class_dirs:
        rumor_file_dir_list.append(os.path.join(target_path, filename, "rumours"))
        non_rumor_file_dir_list.append(os.path.join(target_path, filename, "non-rumours"))

    all_non_rumor_content_file_dir_list = [] #所有谣言评论数据目录（根的下下一级目录）
    all_rumor_content_file_dir_list = []
    all_rumor_srctweet_file_dir_list = [] #所有谣言原文数据目录（根的下下一级目录）
    all_non_rumor_srctweet_file_dir_list = []
    a_rumor_content_dir = []
    a_non_rumor_content_dir = []
    
    for files in rumor_file_dir_list: 
        files_dir = os.listdir(files)
        for file_dir_files in files_dir:
            all_rumor_content_file_dir_list.append(os.path.join(files, file_dir_files, "reactions"))
            all_rumor_srctweet_file_dir_list.append(os.path.join(files, file_dir_files, "source-tweet"))

    for files in non_rumor_file_dir_list:
        file_dir = os.listdir(files)
        for file_dir_files in file_dir:
            all_non_rumor_content_file_dir_list.append(os.path.join(files, file_dir_files, "reactions"))
            all_non_rumor_srctweet_file_dir_list.append(os.path.join(files, file_dir_files, "source-tweet"))
            
    return all_rumor_srctweet_file_dir_list, all_rumor_content_file_dir_list, all_non_rumor_srctweet_file_dir_list, all_non_rumor_content_file_dir_list

In [3]:
target_path = r"D:\NoteBook\RumerDetection\data\pheme-rnr-dataset"
all_rumor_srctweet_list, _, all_non_rumor_srctweet_list, _ = get_data_dir(target_path)
print(all_rumor_srctweet_list[0], all_non_rumor_srctweet_list[0])

D:\NoteBook\RumerDetection\data\pheme-rnr-dataset\charliehebdo\rumours\552783238415265792\source-tweet D:\NoteBook\RumerDetection\data\pheme-rnr-dataset\charliehebdo\non-rumours\552784600502915072\source-tweet


In [4]:
#读取数据，通过文件的路径读取，label为谣言和非谣言的标签
def read_data(data_dir_list, label):
    data = []
    c = 1
    for dir in data_dir_list:
        for filename in os.listdir(dir):
            with open(os.path.join(dir, filename), 'r', encoding='utf8') as f:
                content = f.read()
                if c <= 1:
                    print(content, type(content))
                    c+=1
            data_dict = json.loads(content)
            data.append([data_dict["text"], label])
    return data #data[[content, label]]

In [5]:
#获取谣言数据和非谣言数据，并输出谣言和非谣言的文本数量
all_rumor_data = read_data(all_rumor_srctweet_list, 1)
all_non_rumor_data = read_data(all_non_rumor_srctweet_list, 0)
print("谣言的文本总数：",  len(all_rumor_data))
print("非谣言的文本总数:", len(all_non_rumor_data))

{"contributors":null,"truncated":false,"text":"Breaking: At least 10 dead, 5 injured after tO gunman open fire in offices of Charlie  Hebdo,satirical mag that published Mohammed cartoons","in_reply_to_status_id":null,"id":552783238415265792,"favorite_count":14,"source":"<a href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\">Twitter for iPhone<\/a>","retweeted":false,"coordinates":null,"entities":{"symbols":[],"user_mentions":[],"hashtags":[],"urls":[]},"in_reply_to_screen_name":null,"id_str":"552783238415265792","retweet_count":159,"in_reply_to_user_id":null,"favorited":false,"user":{"follow_request_sent":false,"profile_use_background_image":true,"profile_text_color":"333333","default_profile_image":false,"id":384779793,"profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","verified":false,"profile_location":null,"profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/1576508322\/Henry_Samuel_normal.jpg","profile_sidebar

In [6]:
import unicodedata
import emoji
import unicodedata

#emoji、boxDrawing、Face：https://apps.timwhitlock.info/emoji/tables/unicode#block-6c-other-additional-symbols
import re
# 过滤emoji更全的方法
#pip install emoji
 
def filterEmoji(desstr,restr=' '):
    # 过滤emoji
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)
 
def filterBoxDrawing(desstr, restr=' '):
    # 过滤形如：╠、╤等boxdrawing字符
    co = re.compile(u'[\u2500-\u257f]')
    return co.sub(restr, desstr)
 
def filterFace(desstr, restr= ' '):
    # 过滤：形如[衰]、[生气]、[开心]、[捂脸]等表情，用词典更好些
    p = re.compile('\[.{1,4}\]')
    t = p.findall(desstr)
    for i in t:
        desstr = desstr.replace(i, restr)
    return desstr
 
def filterSpecialSym(desstr, restr=' '):
    #print u'1\u20e3\ufe0f' #10个特殊的类似emoij的表情
    co = re.compile(u'[0-9]?\u20e3\ufe0f?')
    return co.sub(restr, desstr)
 
def bodyNorm(body):
    #body = re.compile(u'''\\\\\\\\\\\\\\\\n''').sub(' ', body) # 得用16个斜杠才行震惊
    body = re.compile(u'''\\\\+?n''').sub(' ', body)
    body = filterSpecialSym(body)
    body = filterEmoji(body)
    body = filterBoxDrawing(body)
    body = filterFace(body)
    return body

ModuleNotFoundError: No module named 'emoji'

# 标点符号和去表情实验

In [7]:
contentbody = "dhadhh 8 * 9.dapa😳😒🙈😳🙈''/..///.1__===1- dadkhh"
#ucontentbody = str(contentbody, 'utf-8')
print('处理前:',contentbody)
body = bodyNorm(contentbody)
print('处理后:',body)
 
import jieba
from string import punctuation as enpunctuation
zhonPunctuation = u'''＂ ＃ ＄ ％ ＆ ＇ （ ） ＊ ＋ ， － ／ ： ； ＜ ＝ ＞ ＠ ［ ＼ ］ ＾ ＿ ｀ ｛ ｜ ｝ ～ ｟ ｠ ｢ ｣ ､  〃 〈 〉 《 》 「 」 『 』 【 】 〔 〕 〖 〗 〘 〙 〚 〛 〜 〝 〞 〟  〾 〿 – — ‘ ’ ‛ “ ” „ ‟ … ‧ ﹏ ﹑ ﹔ · ！ ？ ｡ → 、 。'''
punctuations = set([str(i) for i in enpunctuation]) | set([str(i) for i in zhonPunctuation])
words = [seg for seg in jieba.cut(body) if seg!=' ' and seg not in punctuations]
print('分词后:',' '.join(words))

处理前: dhadhh 8 * 9.dapa😳😒🙈😳🙈''/..///.1__===1- dadkhh
处理后: dhadhh 8 * 9.dapa     ''/..///.1__===1- dadkhh


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\towa\AppData\Local\Temp\jieba.cache
Loading model cost 0.609 seconds.
Prefix dict has been built successfully.


分词后: dhadhh 8 9 dapa .. 1 __ 1 dadkhh


In [8]:
#将数据合并后并打散
all_data = all_rumor_data + all_non_rumor_data
random.shuffle(all_data)

In [9]:
#将数据分割为训练集和测试集,分为带标签和不带标签总共四个文件
def all_data_save(train, all_data, islabel, txtname):
    # 在生成all_data.txt之前，首先将其清空
    filename = train + '_' + txtname
    data_path = './data/' + filename
    with open(data_path, 'w') as f:
        f.seek(0)
        f.truncate()
    count = 0
    punc = '\]~`!#$%^&*()_+-=|\';"":/.,?><~·！@#￥%……&*（）——+-=“：’；、。，？》《{}1234567890→✔️\[✏'
    if islabel == 1 and train == 'train':
        with open(data_path, 'a',encoding='UTF-8') as f:
            for data in  all_data:
                if count > int(0.9 * len(all_data)):#训练集和测试集9:1
                    break
                count += 1
                #print(type(data))
                data = [str(i) for i in data]
                #data[0] = re.sub('@\w+|http.*\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = re.sub('http.*'," URL", data[0]) #将链接替换为URL 
                data[0] = re.sub('@\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = bodyNorm(data[0]) #去表情
                if len(data[0]) > 1:
                    f.write(data[0].strip() + '\t' + data[1] + '\n')
    elif islabel == 1 and train == 'test':
        with open(data_path, 'a',encoding='UTF-8') as f:
            for data in  all_data: 
                count += 1
                if count < int(0.9 * len(all_data)):#训练集和测试集9:1
                    continue
               
                #print(type(data))
                data = [str(i) for i in data]
                #data[0] = re.sub('@\w+|http.*\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = re.sub('http.*'," URL", data[0]) #将链接替换为URL 
                data[0] = re.sub('@\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = bodyNorm(data[0]) #去表情
                if len(data[0]) > 1:
                    f.write(data[0].strip() + '\t' + data[1] + '\n')
   
    if islabel == 0 and train == 'train':
        with open(data_path, 'a',encoding='UTF-8') as f:
            for data in  all_data:
                if count > int(0.9 * len(all_data)):#训练集和测试集9:1
                    break
                count += 1
                #print(type(data))
                data = [str(i) for i in data]
                #data[0] = re.sub('@\w+|http.*\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = re.sub('http.*'," URL", data[0]) #将链接替换为URL 
                data[0] = re.sub('@\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = bodyNorm(data[0]) #去表情
                if len(data[0]) > 1 != "":
                    f.write(data[0].strip() + '\n')
    elif islabel == 0 and train == 'test':
        with open(data_path, 'a',encoding='UTF-8') as f:
            for data in  all_data:
                count += 1
                if count < int(0.9 * len(all_data)):#训练集和测试集9:1
                    continue
                
                #print(type(data))
                data = [str(i) for i in data]
                #data[0] = re.sub('@\w+|http.*\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = re.sub('http.*'," URL", data[0]) #将链接替换为URL 
                data[0] = re.sub('@\w+|\n|[%s]+' %punc,"", data[0])
                data[0] = bodyNorm(data[0]) #去表情
                if len(data[0]) > 1:
                    f.write(data[0].strip() + '\n')

In [10]:
all_data_save("train", all_data, 1, "data.txt")
all_data_save("test", all_data, 1, "data.txt")
all_data_save("train", all_data, 0, "data_non_label.txt")
all_data_save("test", all_data, 0, "data_non_label.txt")

# 分词和去停用词

In [11]:
#分词和去停用词
import pandas as pd
import nltk
from string import punctuation as enpunctuation

In [12]:
with open('./data/train_data_non_label.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()
    # st是sentence的缩写
    #srctweet = [st.split() for st in lines]
    #使用nltk分词
    punctuations = set([str(i) for i in enpunctuation]) | set([str(i) for i in zhonPunctuation]) #去标点符号
    srctweet = [nltk.word_tokenize(st) for st in lines if st != '' and st not in punctuations]
    #srctweet = [nltk.word_tokenize(st) for st in lines]

'# sentences: %d' % len(srctweet) # 输出 '# sentences: 42068'

'# sentences: 1872'

In [13]:
for st in srctweet[:3]:
    print('# tokens:', len(st), st[:5])


# tokens: 9 ['Charlie', 'Hebdos', 'Last', 'Tweet', 'Spoofed']
# tokens: 10 ['Trocadero', 'square', 'in', 'Paris', 'evacuated']
# tokens: 20 ['held', 'by', 'gunman', 'at', 'kosher']


In [14]:
import jieba

In [15]:
stopwords=pd.read_csv("./data/stop_words-master/english.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
stopwords.head()

stopword
0        😳
1        😒
2        🙈
3       😳🙈
4        😬

In [16]:
#清除标点符号
def del_mark(word):
    punc = '~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——+-=“：’；、。，？》《{\]}\[✏'
    return re.sub(r"[%s]+" %punc, "", word)

In [17]:
def drop_stopwords(srctweet,stopwords):
    srctweet_clean = []
    all_words = []
    for line in srctweet:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            word = del_mark(word)
            line_clean.append(word)
            all_words.append(word)##记录所有line_clean中的词
        srctweet_clean.append(line_clean)
    return srctweet_clean,all_words
    #print (contents_clean)
        

stopwords = stopwords.stopword.values.tolist()
srctweet_clean,all_words = drop_stopwords(srctweet,stopwords)

In [18]:
print(srctweet_clean[0]), all_words[0], len(all_words), len(srctweet_clean)

['Charlie', 'Hebdos', 'Last', 'Tweet', 'Spoofed', 'ISIS', 'Leader', 'AlBaghdadi', 'URL']


(None, 'Charlie', 16354, 1872)

In [68]:
for st in srctweet_clean[:3]:
    print('# tokens:', len(st), st[:5])

# tokens: 10 ['Police', 'tear', 'gas', 'protestors', 'row']
# tokens: 11 ['The', 'Eiffel', 'Tower', 'dark', 'local']
# tokens: 9 ['Ottawa', 'shooting', 'Soldier', 'dies', 'injuries']


In [69]:
max_srctweet = 0 #最大评论
for i in srctweet_clean:
    max_srctweet = max(max_srctweet, len(i))
print(max_srctweet)

26


In [70]:
#将all_words保存到txt
def all_words_save_to_txt(all_words, path):
    # 在生成all_data.txt之前，首先将其清空
    with open(path, 'w') as f:
        f.seek(0)
        f.truncate()

    with open(path, 'a',encoding='UTF-8') as f:
        for data in  all_words:
                f.write(data+'\n')

In [23]:
all_words_path = './data/all_words.txt'
all_words_save_to_txt(all_words, all_words_path)

# 建立索引

In [24]:
import collections
import math
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data

In [25]:
counter = collections.Counter([tk for st in srctweet_clean for tk in st])
counter = dict(filter(lambda x: x[1] >= 3, counter.items())) #将出现次数小于3的词去掉

idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)}
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx]
           for st in srctweet]
num_tokens = sum([len(st) for st in dataset])
'# tokens: %d' % num_tokens # 输出 '# tokens: '

'# tokens: 39389'

# 二次采样

In [26]:
def discard(idx):
    return random.uniform(0, 1) < 1 - math.sqrt(
        1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]
'# tokens: %d' % sum([len(st) for st in subsampled_dataset]) # '# tokens: '

'# tokens: 14362'

# 提取中心词和背景词

In [27]:
def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [], []
    for st in dataset:
        if len(st) < 2:  # 每个句子至少要有2个词才可能组成一对“中心词-背景词”
            continue
        centers += st
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size),
                                 min(len(st), center_i + 1 + window_size)))
            indices.remove(center_i)  # 将中心词排除在背景词之外
            contexts.append([st[idx] for idx in indices])
    return centers, contexts

In [28]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset)
for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)


dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1, 2]
center 1 has contexts [0, 2]
center 2 has contexts [0, 1, 3, 4]
center 3 has contexts [1, 2, 4, 5]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [3, 4, 6]
center 6 has contexts [4, 5]
center 7 has contexts [8, 9]
center 8 has contexts [7, 9]
center 9 has contexts [8]


In [29]:
all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

# 负采样

In [30]:
def get_negatives(all_contexts, sampling_weights, K):
    all_negatives, neg_candidates, i = [], [], 0
    population = list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # 根据每个词的权重（sampling_weights）随机生成k个词的索引作为噪声词。
                # 为了高效计算，可以将k设得稍大一点
                i, neg_candidates = 0, random.choices(
                    population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            # 噪声词不能是背景词
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token]
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

In [31]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, centers, contexts, negatives):
        assert len(centers) == len(contexts) == len(negatives)
        self.centers = centers
        self.contexts = contexts
        self.negatives = negatives

    def __getitem__(self, index):
        return (self.centers[index], self.contexts[index], self.negatives[index])

    def __len__(self):
        return len(self.centers)

In [32]:
def batchify(data):
    """用作DataLoader的参数collate_fn: 输入是个长为batchsize的list, 
    list中的每个元素都是Dataset类调用__getitem__得到的结果
    """
    max_len = max(len(c) + len(n) for _, c, n in data)
    centers, contexts_negatives, masks, labels = [], [], [], []
    for center, context, negative in data:
        cur_len = len(context) + len(negative)
        centers += [center]
        contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
        masks += [[1] * cur_len + [0] * (max_len - cur_len)]
        labels += [[1] * len(context) + [0] * (max_len - len(context))]
    return (torch.tensor(centers).view(-1, 1), torch.tensor(contexts_negatives),
            torch.tensor(masks), torch.tensor(labels))


In [33]:
batch_size = 512
num_workers = 0 if sys.platform.startswith('win32') else 4

dataset = MyDataset(all_centers, 
                    all_contexts, 
                    all_negatives)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True,
                            collate_fn=batchify, 
                            num_workers=num_workers)
for batch in data_iter:
    for name, data in zip(['centers', 'contexts_negatives', 'masks',
                           'labels'], batch):
        print(name, 'shape:', data.shape)
    break


centers shape: torch.Size([512, 1])
contexts_negatives shape: torch.Size([512, 42])
masks shape: torch.Size([512, 42])
labels shape: torch.Size([512, 42])


In [34]:
embed = nn.Embedding(num_embeddings=20, embedding_dim=4)
embed.weight


Parameter containing:
tensor([[-1.7168,  1.5454,  1.5316,  0.5437],
        [ 0.4663, -0.7534,  0.4916,  0.0569],
        [-0.3986,  0.9203,  0.1550,  0.1650],
        [-0.1046, -0.4030, -0.5459, -0.3333],
        [ 0.4725,  0.3025,  1.6655,  0.4323],
        [-1.1341,  1.8225, -0.3170, -1.0209],
        [ 0.3295,  0.9251,  2.5884,  0.1076],
        [-0.7215,  1.7432,  1.3368,  1.3773],
        [ 0.6271,  0.5001,  0.3959, -0.0629],
        [ 0.2785,  0.9637, -1.4650,  0.3057],
        [-0.3053, -0.6449,  0.7515,  0.7903],
        [ 0.6640, -0.8161, -0.7772, -1.6631],
        [-1.7117, -1.2093,  1.7904,  0.0648],
        [ 0.2697,  1.5622, -0.5857, -0.5732],
        [ 1.5137, -0.9952,  0.4684, -0.7182],
        [-1.0255, -0.4127,  0.5263, -0.0618],
        [ 0.7937,  0.4219,  1.6367,  0.3985],
        [ 0.8639, -0.0248,  0.2762,  0.0393],
        [-0.9568,  2.9950, -0.5155,  0.2228],
        [ 0.2126,  1.8832, -0.5369, -0.8803]], requires_grad=True)

In [35]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.long)
embed(x)


tensor([[[ 0.4663, -0.7534,  0.4916,  0.0569],
         [-0.3986,  0.9203,  0.1550,  0.1650],
         [-0.1046, -0.4030, -0.5459, -0.3333]],

        [[ 0.4725,  0.3025,  1.6655,  0.4323],
         [-1.1341,  1.8225, -0.3170, -1.0209],
         [ 0.3295,  0.9251,  2.5884,  0.1076]]], grad_fn=<EmbeddingBackward>)

# 小批量乘法

In [36]:
X = torch.ones((2, 1, 4))
Y = torch.ones((2, 4, 6))
torch.bmm(X, Y).shape


torch.Size([2, 1, 6])

# 跳字模型向前计算

In [37]:
def skip_gram(center, contexts_and_negatives, embed_v, embed_u):
    v = embed_v(center)
    u = embed_u(contexts_and_negatives)
    pred = torch.bmm(v, u.permute(0, 2, 1))
    return pred


# 训练模型

# 交叉熵损失函数

In [38]:
class SigmoidBinaryCrossEntropyLoss(nn.Module):
    def __init__(self): # none mean sum
        super(SigmoidBinaryCrossEntropyLoss, self).__init__()
    def forward(self, inputs, targets, mask=None):
        """
        input – Tensor shape: (batch_size, len)
        target – Tensor of the same shape as input
        """
        inputs, targets, mask = inputs.float(), targets.float(), mask.float()
        res = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction="none", weight=mask)
        return res.mean(dim=1)

loss = SigmoidBinaryCrossEntropyLoss()


In [39]:
pred = torch.tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]])
# 标签变量label中的1和0分别代表背景词和噪声词
label = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0]])
mask = torch.tensor([[1, 1, 1, 1], [1, 1, 1, 0]])  # 掩码变量
loss(pred, label, mask) * mask.shape[1] / mask.float().sum(dim=1)


tensor([0.8740, 1.2100])

In [40]:
def sigmd(x):
    return - math.log(1 / (1 + math.exp(-x)))

print('%.4f' % ((sigmd(1.5) + sigmd(-0.3) + sigmd(1) + sigmd(-2)) / 4)) # 注意1-sigmoid(x) = sigmoid(-x)
print('%.4f' % ((sigmd(1.1) + sigmd(-0.6) + sigmd(-2.2)) / 3))


0.8740
1.2100


# 初始化模型参数

In [41]:
embed_size = 100
net = nn.Sequential(
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size),
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size)
)


# 定义训练函数

In [71]:
def train(net, lr, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("train on", device)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        start, l_sum, n = time.time(), 0.0, 0
        for batch in data_iter:
            center, context_negative, mask, label = [d.to(device) for d in batch]

            pred = skip_gram(center, context_negative, net[0], net[1])

            # 使用掩码变量mask来避免填充项对损失函数计算的影响
            l = (loss(pred.view(label.shape), label, mask) *
                 mask.shape[1] / mask.float().sum(dim=1)).mean() # 一个batch的平均loss
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.cpu().item()
            n += 1
        print('epoch %d, loss %.2f, time %.2fs'
              % (epoch + 1, l_sum / n, time.time() - start))


In [43]:
#开始训练
train(net, 0.008, 10)

train on cuda
epoch 1, loss 3.94, time 0.48s
epoch 2, loss 3.13, time 0.19s
epoch 3, loss 2.53, time 0.19s
epoch 4, loss 2.05, time 0.19s
epoch 5, loss 1.66, time 0.19s
epoch 6, loss 1.34, time 0.19s
epoch 7, loss 1.09, time 0.20s
epoch 8, loss 0.88, time 0.19s
epoch 9, loss 0.72, time 0.19s
epoch 10, loss 0.59, time 0.20s


In [44]:
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    print(query_token + "词嵌入:", x.shape, "\n")
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x) / (torch.sum(W * W, dim=1) * torch.sum(x * x) + 1e-9).sqrt()
    _, topk = torch.topk(cos, k=k+1)
    topk = topk.cpu().numpy()
    for i in topk[1:]:  # 除去输入词
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))

get_similar_tokens('happening', 3, net[0])

happening词嵌入: torch.Size([100]) 

cosine sim=0.359: Eiffel
cosine sim=0.337: Cpl
cosine sim=0.335: Ukraine


# 保存模型

In [ ]:
d1 = {
    'en':'英语',
    'cn':'中文',
    'fr':'法语',
    'jp':'日语'
}
l1 = list(d1.items())
d1

In [46]:
token_to_idx_list = list(token_to_idx.items()) #将词典转换为list
print(token_to_idx_list[0])
with open("./data/token_to_idx.txt", 'w') as f:
    f.seek(0)
    f.truncate()
with open("./data/token_to_idx.txt", 'a', encoding='utf8') as f:
    for i in token_to_idx_list:
        f.write(i[0] + "\t" + str(i[1]) + '\n')
    

('LATEST', 0)


# 保存模型

In [47]:
#保存模型embedding
torch.save(net, "./data/net.pt")

# 加载模型

In [ ]:
embed_size = 100
net1 = nn.Sequential(
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size),
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size)
)
net1 = torch.load("./data/net.pt")
net1.eval()

In [ ]:
type(net[0])

In [ ]:
for param_tensor in net[0].state_dict():
    print(param_tensor, 
          "\t", 
          net[0].state_dict()[param_tensor].size()
         )

In [ ]:
print(net[0])

In [ ]:
print(net[0].weight.data[token_to_idx["Germanwings"]])

In [ ]:
type(net[0].weight.data)

In [ ]:
net[0].weight.data.shape

In [ ]:
torch.save(net.state_dict(), "net_params.pkl")

In [ ]:
print(torch.load('net_params.pkl'))